In [6]:
import os
import tqdm
import sys
import keras

In [ ]:
LENGTH=10 #in secs
FEATURE_DIM = 1582 # opensmile feature dimension for an input chunk

In [8]:
# USER INPUT- features_dir
# features_dir- path to save or load vgg face features
curr=os.getcwd()
repo_path=curr.split('/code')[0]
data_path=repo_path+'/DIFv2'
features_path=repo_path+'/features'#  Folder containing all features audio and video
features_dir=features_path+'/'+str(LENGTH)+'/opensmile'# Path to save/load opensmile features


# Generator and data split code

In [8]:
import csv
'''
Input-  csv_file
Output- partition train, val test. Each partition consists of list of .npy files and dictionary of labels.
'''
def train_test_split(csv_path):
    label={'Drunk':1, 'Sober':0}
    partition={}
    train={}
    val={}
    test={}
    
    train_list=[]
    val_list=[]
    test_list=[]
    train_label={}
    val_label={}
    test_label={}
    
    with open(csv_path) as csvfile:
        reader=csv.reader(csvfile,delimiter=',')
        for row in reader:
            filename=row[2]
            filename=filename[:-4]
            if row[0]=='train':
                train_label[filename]=label[row[1]]
                train_list.append(filename)
            elif row[0]=='val':
                val_label[filename]=label[row[1]]
                val_list.append(filename)
            elif row[0]=='test':
                test_label[filename]=label[row[1]]
                test_list.append(filename)
            else:
                print("Error in label")
                return None
    train['list']=train_list
    val['list']=val_list
    test['list']=test_list
    
    train['label']=train_label
    val['label']=val_label
    test['label']=test_label
    
    partition['train']=train
    partition['val']=val
    partition['test']=test
    
    return partition

def count_classes(d):
    values=list(d.values())
    zeros=values.count(0)
    return (zeros,len(values)-zeros)

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels,datapath, batch_size=32, dim=(1582),n_classes=2, shuffle=True):
        'Initialization'        
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        #self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.path=datapath

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))


    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        import gc
        gc.collect()
        return X, y
    
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load(self.path+'/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

# Build Model

3.1 Model creation and summary

ordering of batch norm and dropout
https://stackoverflow.com/questions/39691902/ordering-of-batch-normalization-and-dropout

In [2]:
from keras.optimizers import Adam
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense, Input, Dropout, LSTM, Activation,BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import keras
from time import time
import gc
import numpy as np
keras.backend.clear_session()

Using TensorFlow backend.


In [1]:
def create_model(num_class,dense1_units,dense2_units,dropout,input_shape=(1582,)):
    """
        Two layer DNN
    """
    X=Input(shape=input_shape)
    norm=BatchNormalization()(X)
    layer1=Dense(dense1_units, activation='relu')(norm)
    drop=Dropout(rate=dropout)(layer1)
    layer2=Dense(dense2_units, activation='relu')(drop)
    drop=Dropout(rate=dropout)(layer2)
    prob=Dense(num_class, activation='sigmoid')(layer2)
    return Model(inputs = X, outputs = prob)

def basic_model(num_class,input_shape=(1582,)):
    """
        Single layer DNN
    """
    X=Input(shape=input_shape)
    prob=Dense(num_class, activation='sigmoid')(X)
    return Model(inputs = X, outputs = prob)
    
    

In [9]:
dense1=256
dense2=128
#dense3=128
dropout=.2
hp=2
class_num=2
model=create_model(class_num,dense1,dense2,dropout,input_shape=(FEATURE_DIM,))
#USER INPUT, path to save/ load model
model_path=repo_path+'/saved_models/audio_open/'+str(LENGTH)+'/hp'+str(hp)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1582)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1582)              6328      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               405248    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 444,730
Trainable params: 441,566
Non-trainable params: 3,164
_________________________________________________

In [21]:
def load_keras_model(path):
    if os.path.isfile(path):
        return load_model(path)
#Loading data filenames split
#USER INPUT
partition=train_test_split(repo_path+'/DIFv2/10/train_test_sets/1/split_4540_642_948.csv')# or enter path to the split.csv in the parent directory 
print("Number of training examples ")
print(len(partition['train']['list']))
print("Number of validation examples ")
print(len(partition['val']['list']))

params = {'datapath':features_dir ,
          'dim': (FEATURE_DIM,),
          'batch_size': 64,
          'n_classes': 2,
          'shuffle': True}
    
#weights for imbalance classes
count=count_classes(partition['train']['label'])
print("Class instances in training class.\n Sober:",count[0]," Drunk:",count[1])
weight_0=float(count[0]+count[1])/float(count[0])
weight_1=float(count[0]+count[1])/float(count[1])
class_weight={0:weight_0, 1:weight_1}

#instances in val set
count=count_classes(partition['val']['label'])
print("Class instances in val class.\n Sober:",count[0]," Drunk:",count[1])

#instances in test set
count=count_classes(partition['test']['label'])
print("Class instances in test class.\n Sober:",count[0]," Drunk:",count[1])


Number of training examples 
4540
Number of validation examples 
642
Class instances in training class.
 Sober: 1045  Drunk: 3495
Class instances in val class.
 Sober: 321  Drunk: 321
Class instances in test class.
 Sober: 306  Drunk: 642


In [22]:

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=["accuracy"])

#saving best model
checkpoint = ModelCheckpoint(model_path+'/model-{epoch:03d}-{val_acc:03f}.h5', verbose=1, monitor='val_acc',save_best_only=False, mode='max',period=5)


#tensorboard
tensorboard = TensorBoard(log_dir=model_path+"/log/{}".format(time()))

train_generator=DataGenerator(partition['train']['list'],partition['train']['label'], **params)
val_generator=DataGenerator(partition['val']['list'],partition['val']['label'], **params)
print("generator created")
model.fit_generator(generator=train_generator,epochs=80,validation_data=val_generator,
                    use_multiprocessing=True,
                    workers=6,callbacks=[checkpoint,tensorboard],class_weight=class_weight)

generator created
Epoch 1/80
70/70 [==============================] - 2s 35ms/step - loss: 0.3773 - acc: 0.9198 - val_loss: 0.5590 - val_acc: 0.8383
Epoch 2/80
70/70 [==============================] - 2s 34ms/step - loss: 0.1730 - acc: 0.9642 - val_loss: 0.4436 - val_acc: 0.8625
Epoch 3/80
70/70 [==============================] - 2s 34ms/step - loss: 0.1217 - acc: 0.9732 - val_loss: 0.8580 - val_acc: 0.8234
Epoch 4/80
70/70 [==============================] - 2s 35ms/step - loss: 0.0912 - acc: 0.9813 - val_loss: 0.9090 - val_acc: 0.8352
Epoch 5/80
70/70 [==============================] - 3s 37ms/step - loss: 0.0889 - acc: 0.9809 - val_loss: 1.0475 - val_acc: 0.8266

Epoch 00005: saving model to /home/ubuntu/Vineet/repo/saved_models/audio_open/10/hp2/model-005-0.826562.h5
Epoch 6/80
70/70 [==============================] - 2s 24ms/step - loss: 0.0701 - acc: 0.9858 - val_loss: 0.9453 - val_acc: 0.8313
Epoch 7/80
70/70 [==============================] - 3s 36ms/step - loss: 0.0701 - acc: 0

Epoch 54/80
70/70 [==============================] - 2s 34ms/step - loss: 0.0269 - acc: 0.9952 - val_loss: 1.7154 - val_acc: 0.8391
Epoch 55/80
70/70 [==============================] - 2s 33ms/step - loss: 0.0203 - acc: 0.9959 - val_loss: 1.7939 - val_acc: 0.8406

Epoch 00055: saving model to /home/ubuntu/Vineet/repo/saved_models/audio_open/10/hp2/model-055-0.840625.h5
Epoch 56/80
70/70 [==============================] - 2s 35ms/step - loss: 0.0075 - acc: 0.9978 - val_loss: 1.7387 - val_acc: 0.8453
Epoch 57/80
70/70 [==============================] - 3s 36ms/step - loss: 0.0097 - acc: 0.9982 - val_loss: 1.8909 - val_acc: 0.8289
Epoch 58/80
70/70 [==============================] - 2s 35ms/step - loss: 0.0149 - acc: 0.9975 - val_loss: 1.6969 - val_acc: 0.8375
Epoch 59/80
70/70 [==============================] - 2s 33ms/step - loss: 0.0060 - acc: 0.9989 - val_loss: 1.7592 - val_acc: 0.8438
Epoch 60/80
70/70 [==============================] - 2s 31ms/step - loss: 0.0038 - acc: 0.9988 - val

# Basic model

In [27]:

hp=3
class_num=2
model=basic_model(class_num,input_shape=(FEATURE_DIM,))
#USER INPUT, path to save/ load model
model_path=repo_path+'/saved_models/audio_open/'+str(LENGTH)+'/hp'+str(hp)
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 1582)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 3166      
Total params: 3,166
Trainable params: 3,166
Non-trainable params: 0
_________________________________________________________________


In [28]:

partition=train_test_split(repo_path+'/DIFv2/10/train_test_sets/1/split_4540_642_948.csv')
print("Number of training examples ")
print(len(partition['train']['list']))
print("Number of validation examples ")
print(len(partition['val']['list']))

params = {'datapath':features_dir ,
          'dim': (FEATURE_DIM,),
          'batch_size': 64,
          'n_classes': 2,
          'shuffle': True}
    
#weights for imbalance classes
count=count_classes(partition['train']['label'])
print("Class instances in training class.\n Sober:",count[0]," Drunk:",count[1])
weight_0=float(count[0]+count[1])/float(count[0])
weight_1=float(count[0]+count[1])/float(count[1])
class_weight={0:weight_0, 1:weight_1}

#instances in val set
count=count_classes(partition['val']['label'])
print("Class instances in val class.\n Sober:",count[0]," Drunk:",count[1])

#instances in test set
count=count_classes(partition['test']['label'])
print("Class instances in test class.\n Sober:",count[0]," Drunk:",count[1])


Number of training examples 
4540
Number of validation examples 
642
Class instances in training class.
 Sober: 1045  Drunk: 3495
Class instances in val class.
 Sober: 321  Drunk: 321
Class instances in test class.
 Sober: 306  Drunk: 642


In [29]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=["accuracy"])

#saving best model
checkpoint = ModelCheckpoint(model_path+'/model-{epoch:03d}-{val_acc:03f}.h5', verbose=1, monitor='val_acc',save_best_only=False, mode='max',period=5)


#tensorboard
tensorboard = TensorBoard(log_dir=model_path+"/log/{}".format(time()))

train_generator=DataGenerator(partition['train']['list'],partition['train']['label'], **params)
val_generator=DataGenerator(partition['val']['list'],partition['val']['label'], **params)
print("generator created")
model.fit_generator(generator=train_generator,epochs=80,validation_data=val_generator,
                    use_multiprocessing=True,
                    workers=6,callbacks=[checkpoint,tensorboard],class_weight=class_weight)

generator created
Epoch 1/80
70/70 [==============================] - 2s 35ms/step - loss: 14.9692 - acc: 0.6919 - val_loss: 7.0809 - val_acc: 0.5492
Epoch 2/80
70/70 [==============================] - 2s 34ms/step - loss: 12.9183 - acc: 0.7188 - val_loss: 6.8895 - val_acc: 0.5570
Epoch 3/80
70/70 [==============================] - 3s 38ms/step - loss: 11.8630 - acc: 0.7583 - val_loss: 6.4972 - val_acc: 0.5867
Epoch 4/80
70/70 [==============================] - 2s 36ms/step - loss: 11.0016 - acc: 0.7712 - val_loss: 6.4710 - val_acc: 0.5867
Epoch 5/80
70/70 [==============================] - 3s 37ms/step - loss: 10.6668 - acc: 0.7891 - val_loss: 6.2348 - val_acc: 0.6023

Epoch 00005: saving model to /home/ubuntu/Vineet/repo/saved_models/audio_open/10/hp3/model-005-0.602344.h5
Epoch 6/80
70/70 [==============================] - 2s 23ms/step - loss: 10.3968 - acc: 0.7900 - val_loss: 6.2256 - val_acc: 0.5984
Epoch 7/80
70/70 [==============================] - 3s 36ms/step - loss: 11.6091 -